## Sternberg remapping of columns

The purpose of the remapping is to provide a finished event file `_events_temp4.tsv`
by remapping the corrected BIDS events files `_events_temp2.tsv'.

1. Run remapping of `value` to `event_type`, `task_role`, and `letter`.
2. Set the `trial` column based on the `ready` value in the `task_role`.
3. Count the number of targets in each trial and set the `memory_cond`.
4. Change the following `task_role` items based on contents:
  1. `remembered_correct` or `ignored_correct` to `continue` if last item in trial.
  2. `probe_target` to `probe_nontarget` for probes corresponding to letters ignored in that trial.
  3. `probe_target` to `probe_not_shown` for probes corresponding to letters not shown in that trial.
  4. `remembered_correct` to `remembered_incorrect` to indicate right button but should have been left.
  5. `ignored_correct` to `ignored_incorrect` to indicate left button but should have been right.
  6. `bad_trial` if number of events in the trial is not 14 or 13. (Also check that if 13,
it is the feedback event that is missing.)

In [1]:
import os
from hed.tools import BidsTsvDictionary, HedLogger
from hed.util import get_file_list, get_new_dataframe

def set_trials(df, status_record, file_key):
    # Set the trials column based on start of trial marker value==ready
    df['trial'] = 'n/a'
    trial_num = 0
    for index, value in df['event_type'].iteritems():
        if value == 'show_cross':
            trial_num += 1
        df.loc[index, 'trial'] = trial_num
    status_record.add(file_key, "Add trial number column")


def set_memory_cond(df, status_record, file_key):
    # Set the memory_cond to number of targets in the trial
    df['memory_cond'] = 0
    max_trial = df['trial'].max()
    target_mask = df['task_role'].map(str) == 'to_remember'
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        df.loc[trial_mask, 'memory_cond'] =  sum(trial_mask & target_mask)
    status_record.add(file_key, f"Set the memory condition column")


def set_probe(df, status_record, file_key):
    # Set the task_role=probe_nontarget if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    probe_mask = df['task_role'] == 'probe_target'
    target_mask = df['task_role'] == 'target'
    nontarget_mask = df['task_role'] == 'nontarget'
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        target_set = set(df.loc[trial_mask & target_mask, 'letter'])
        nontarget_set = set(df.loc[trial_mask & nontarget_mask, 'letter'])
        next_mask = trial_mask & probe_mask
        if not sum(next_mask):
            status_record.add(file_key, f"The trial {index} does not have a probe letter",
                              level="WARNING", also_print=True)
            continue
        probe_letter = df.loc[next_mask, 'letter'].values[0]
        if probe_letter in target_set:
            continue
        elif probe_letter in nontarget_set:
            df.loc[trial_mask & probe_mask, 'task_role'] = 'probe_nontarget'
        else:
            df.loc[trial_mask & probe_mask, 'task_role'] = 'probe_not_shown'
    status_record.add(file_key,
                      f"Set the task_role of the probe to one of 'probe_target, probe_nontarget, probe_not_shown")


def set_continue(df, status_record, file_key):
    # Set the last event in each trial to have task_role=continue
    max_trial = df['trial'].max()
    for index in range(1, max_trial+1):
        df_col = df.loc[df['trial'] == index, 'task_role']
        index = df_col.index.values
        df.loc[index[-1], 'task_role'] = 'continue'
    status_record.add(file_key, f"Set the last event in each trial to have the task_role of continue")


def set_response(df, status_record, file_key):
    # Set the task_role=in_group_correct if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    probe_in_mask = df['task_role'] == 'probe_target'
    probe_out_mask = (df['task_role'] == 'probe_nontarget') | (df['task_role'] == 'probe_out')
    right_mask = (df['event_type'] == 'right_click') & (df['task_role'] == 'in_group_correct')
    left_mask = (df['event_type'] == 'left_click') & (df['task_role'] == 'out_group_correct')
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        out_group_incorrect = trial_mask & probe_in_mask & left_mask
        in_group_incorrect = trial_mask & probe_out_mask & right_mask

        if sum(out_group_incorrect):
            df.loc[out_group_incorrect, 'task_role'] = 'out_group_incorrect'
        elif sum(in_group_incorrect):
            df.loc[in_group_incorrect, 'task_role'] = 'in_group_incorrect'
    status_record.add(file_key, f"Set response indicating correct or incorrect response")

def set_bad_trials(df, status_record, file_key):
    # Set the task_role=in_group_correct if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    feedback_mask = (df['event_type'] == 'sound_beep') | (df['event_type'] == 'sound_buzz')
    for index in range(1, max_trial+1):
        trial_mask = df['trial'] == index
        if sum(trial_mask) == 14:
            continue
        elif sum(trial_mask) == 13 and not sum(feedback_mask & trial_mask):
            status_record.add(file_key, f"Trial {index} has no feedback event -- bad trial", level="WARNING")
        else:
            df.loc[df['trial'] == index, 'task_role'] = 'bad_trial'
            status_record.add(file_key, f"Trial {index} has {sum(df['trial'] == index)} -- bad trial", level="WARNING")

# Set the specific variables for the Sternberg dataset.
bids_root_path = 'G:\Sternberg\SternbergWorking'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
col_order = ['onset', 'duration', 'sample', 'event_type', 'task_role', 'letter', 'trial', 'memory_cond', 'value']
log_file_name = 'sternberg_05_refactor_remapped_log.json'

# Set up the logger and create the file dictionaries
logger = HedLogger()
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], exclude_dirs = exclude_dirs, name_suffix='_eventstemp2')
bids_dict = BidsTsvDictionary("Bids event files", bids_files, entities=entities)

# Process the data files
for key, file in bids_dict.iter_files():
    this_path = bids_dict.get_file_path(key)
    df_bids = get_new_dataframe(this_path)

    # Set the trials and  memory_cond columns
    set_trials(df_bids, logger, key)
    set_memory_cond(df_bids, logger, key)

    # Correct the probe column values to account for non-target or missing probes
    set_probe(df_bids, logger, key)

    # Set task_role as continue for last click in trial
    set_continue(df_bids, logger, key)

    # Set task_role to indicate the correct response
    set_response(df_bids, logger, key)

    # Set bad trails if trial does not have 14 events
    set_bad_trials(df_bids, logger, key)

    # Save the new events file
    df_bids = df_bids.reindex(columns=col_order)
    logger.add(key, f"Reindexed to column order {str(col_order)}")
    filename = this_path[:-5] + "3.tsv"
    logger.add(key, f"Saved the file to {os.path.basename(filename)}")
    df_bids.to_csv(filename, sep='\t', index=False)

# Output and save the log
logger.print_log()
save_path = os.path.join(bids_root_path, 'code')
logger.save_log(save_path, log_name=log_file_name)

# Output errors
print("\nERROR Summary:")
logger.print_log(level="ERROR")

# Output warnings
print("\nWARNING Summary:")
logger.print_log(level="WARNING")

sub-004_ses-01_run-2 [WARNING]: The trial 1 does not have a probe letter
sub-006_ses-01_run-1 [WARNING]: The trial 1 does not have a probe letter
sub-006_ses-01_run-1 [WARNING]: The trial 2 does not have a probe letter
sub-006_ses-01_run-1 [WARNING]: The trial 4 does not have a probe letter
sub-006_ses-01_run-1 [WARNING]: The trial 5 does not have a probe letter
sub-008_ses-01_run-1 [WARNING]: The trial 1 does not have a probe letter
sub-014_ses-01_run-1 [WARNING]: The trial 11 does not have a probe letter
sub-014_ses-01_run-2 [WARNING]: The trial 3 does not have a probe letter
sub-014_ses-01_run-2 [WARNING]: The trial 8 does not have a probe letter
sub-014_ses-01_run-3 [WARNING]: The trial 22 does not have a probe letter
sub-015_ses-01_run-3 [WARNING]: The trial 25 does not have a probe letter
sub-017_ses-01_run-3 [WARNING]: The trial 3 does not have a probe letter
sub-017_ses-01_run-3 [WARNING]: The trial 5 does not have a probe letter
sub-022_ses-01_run-6 [WARNING]: The trial 6 does